In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import io
import seaborn as sns
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [3]:
data = np.loadtxt("alkan/ALKAN.BLD")

In [4]:
data

array([[ 2.,  1.,  0., ...,  2.,  1.,  1.],
       [ 2.,  0.,  2., ...,  5.,  2.,  1.],
       [ 2.,  0.,  2., ..., 10.,  4.,  2.],
       ...,
       [ 6.,  0.,  0., ..., 31., 13.,  2.],
       [ 6.,  0.,  0., ..., 32., 14.,  2.],
       [ 6.,  0.,  0., ..., 30., 12.,  2.]])

In [5]:
data.shape

(74, 210)

In [6]:
n = data.shape[0]
m = data.shape[1]

In [7]:
(n,m)

(74, 210)

In [8]:
l = []
word = 'A8 '
with open('alkan/set-str/ALKAN.SET') as file:
    for line in file:
        if (word in line):
            ape = float(line.split()[1])
            l.append(ape)

In [9]:
len(l)

74

In [10]:
index_del = []
y = np.array(l)

In [11]:
for i in range(y.shape[0]):
    if abs(y[i] - 0.0) < 1e-10:
        index_del.append(i)

In [12]:
new_data = np.delete(data, index_del, axis=0)
new_y = np.delete(y, index_del, axis=0)

In [13]:
print(new_data.shape)
print(new_y.shape)

(56, 210)
(56,)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(new_data, new_y, test_size=0.3, train_size=0.7)

In [15]:
def R_rise_2 (Eps, y, y_mean):
    result1 = np.sum(Eps**2)
    result2 = np.sum((y-y.mean())**2)
    return 1 - result1 / result2


def Correlation(X, Y):
    X_mean = X.sum() / X.shape[0]
    Y_mean = Y.sum() / Y.shape[0]
    res = ((X) * (Y)).sum()
    res1 = (((X) ** 2)) 
    res2 = (((Y) ** 2)) 
    
    return  res / (((res1.sum() * res2.sum())) ** 0.5)

In [16]:
size_buf = 10
num_selections = 10
min_correlation = 0.95
#100
#15

In [17]:
model = Ridge(alpha=0.2)

In [18]:
y_mean = data.sum(axis = 1)
y_mean = y_mean / m
y_mean

array([0.09285714, 0.1907381 , 0.33778095, 0.31322381, 0.53822857,
       0.50626667, 0.46634762, 0.79669048, 0.74421905, 0.74644286,
       0.7040619 , 0.69749048, 1.11633333, 1.06962381, 1.06011905,
       1.02632381, 0.99708095, 0.99303333, 1.00302857, 0.9837381 ,
       0.93640952, 1.494     , 1.42669524, 1.41860952, 1.43021905,
       1.40549524, 1.36054762, 1.37269524, 1.3561    , 1.37696667,
       1.35222857, 1.3351619 , 1.32382381, 1.3034619 , 1.2579381 ,
       1.27928095, 1.27077619, 1.29133333, 1.18765238, 1.95123333,
       1.87832857, 1.85914286, 1.87329524, 1.82781905, 1.83665714,
       1.78150476, 1.79542857, 1.79074286, 1.78900476, 1.79551905,
       1.7769    , 1.77144762, 1.76990952, 1.77332857, 1.76851429,
       1.76265714, 1.73765238, 1.7309    , 1.70322857, 1.6973619 ,
       1.72905238, 1.70461429, 1.6983381 , 1.73022857, 1.69916667,
       1.66721429, 1.67419048, 1.63922857, 1.64113333, 1.67244286,
       1.5761619 , 1.60750476, 1.60711429, 1.60879524])

In [19]:
buf_r = np.array([0.0 for i in range(size_buf)])
buf_size_ar = np.array([0 for i in range(num_selections)])
iter_buf = 0

buf_numbers = np.array([[-1 for i in range(size_buf)] for k in range(num_selections)])
buf_numbers_1 = np.array([[-1 for i in range(size_buf)] for k in range(num_selections)])

buf = np.array([[[float(i) for i in range(size_buf)] for j in range(n)] for k in range(num_selections)])

X1 = np.array([[float(i) for i in range(1)] for j in range(n)])
X2 = np.array([[float(i) for i in range(2)] for j in range(n)])
buf.shape

(10, 74, 10)

In [20]:
%%time
for i in range(m):
    X1[:,0] = data[:,i]
    model.fit(X1, y)
    if iter_buf < size_buf:
        tmp = model.predict(X1)
        ar = np.array([])

        for j in range (iter_buf):
            t = Correlation (buf[0, :,j], tmp)
            ar = np.append (ar, t)
        #print(ar)
        if (iter_buf == 0) or (ar.max() < min_correlation):    
            buf_r[iter_buf] = R_rise_2(y - tmp, y, y_mean)
            buf_numbers[0, iter_buf] = i
            buf[0, :, iter_buf] = tmp[:]
            iter_buf += 1
            
    else:
        tmp = model.predict(X1)
        ind = buf_r.argmin()
        ar = np.array([])
        for j in range (buf.shape[2]):
            t = Correlation (buf[0, :,j], tmp)
            ar = np.append (ar, t)
        #print(ar)
        if (ar.max() < min_correlation) and (buf_r[ind] < R_rise_2(y - tmp, y, y_mean)): 
            buf_r[ind] = R_rise_2(y - tmp, y, y_mean)
            buf_numbers[0, ind] = i
            buf[0, :, ind] = tmp[:]
buf_numbers_1[0, :] = -2
print(buf_numbers_1, end="\n\n\n\n")
buf_size_ar[0] = iter_buf
print(buf_size_ar[0])

[[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]



1
CPU times: user 81.2 ms, sys: 53 µs, total: 81.3 ms
Wall time: 81.9 ms


In [21]:
%%time
for k in range (1, num_selections):
    iter_buf = 0
    for i in range(buf_size_ar[k - 1]):
        for j in range(0, m):
            #print(data[:,i].shape)
            X2[:, 0] = buf[k - 1, :, i]
            X2[:, 1] = data[:, j]
            model.fit(X2, y)
            if iter_buf < size_buf:
                tmp = model.predict(X2)
                ar = np.array([])
            
                for j1 in range (iter_buf):
                    t = Correlation (buf[0, :,j1], tmp)
                    ar = np.append (ar, t)
                #print(j, ar)
                
                if (iter_buf == 0) or (ar.max() < min_correlation):
                    buf_r[iter_buf] = R_rise_2(y - tmp, y, y_mean)
                    buf_numbers[k, iter_buf] = j
                    #print("*******",i, j)
                    buf_numbers_1[k, iter_buf] = i
                    buf[k, :,iter_buf] = tmp[:]
                    iter_buf += 1

            else:
                tmp = model.predict(X2)
                ind = buf_r.argmin()
                ar = np.array([])
                for j1 in range (buf.shape[2]):
                    t = Correlation (buf[0, :,j1], tmp)
                    ar = np.append (ar, t)
                if (ar.max() < min_correlation) and (buf_r[ind] < R_rise_2(y - tmp, y, y_mean)):
                    buf_r[ind] = R_rise_2(y - tmp, y, y_mean)
                    buf_numbers[k, ind] = j
                    buf_numbers_1[k, ind] = i
                    buf[k, :, ind] = tmp[:]
    buf_size_ar[k] = iter_buf

    print("selection", k + 1, "from", num_selections)

selection 2 from 10
selection 3 from 10
selection 4 from 10
selection 5 from 10
selection 6 from 10
selection 7 from 10
selection 8 from 10
selection 9 from 10
selection 10 from 10
CPU times: user 725 ms, sys: 0 ns, total: 725 ms
Wall time: 743 ms


In [22]:
buf_numbers

array([[ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [23]:
buf_numbers_1

array([[-2, -2, -2, -2, -2, -2, -2, -2, -2, -2],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [24]:
lis = []
for i in range(buf_size_ar[num_selections - 1]):
    lis.append([buf_numbers[num_selections - 1][i]])

In [25]:
for i in range(num_selections - 1, 0, -1):
    #print(i)
    for j in range(buf_size_ar[i - 1]):
        lis[j].append(buf_numbers[i - 1][buf_numbers_1[i,j]])


In [26]:

lis

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [42]:
clf = Ridge(alpha=0.1)
X_train, X_test, y_train, y_test = train_test_split(new_data, new_y, train_size=0.6, test_size=0.4)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

-0.6212283057764496

In [43]:
for i in range(buf_size_ar[num_selections-1]):
    clf.fit(X_train[:,np.unique(lis[i])], y_train)
    print(clf.score(X_test[:,np.unique(lis[i])], y_test))

0.01651920224706771


In [44]:
clf_Lin = LinearRegression()
clf_MGUA = LinearRegression()
res_MGUA = clf_MGUA.fit(X_train[:,np.unique(lis[0])], y_train)
res_Lin = clf_Lin.fit(X_train, y_train)

In [45]:
predict_MGUA = clf_MGUA.predict(X_test[:,np.unique(lis[0])])

In [46]:
predict_Lin = clf_Lin.predict(X_test)

In [48]:
for i in range(predict_Lin.shape[0]):
    print("МГУА", predict_MGUA[i], "Греб.рег", predict_Lin[i], "Истинное значение", y_test[i])

МГУА -127.2542817679558 Греб.рег -123.68835603840671 Истинное значение -129.72
МГУА -127.2542817679558 Греб.рег -120.75691499557331 Истинное значение -95.35
МГУА -96.67937154696133 Греб.рег -128.99122537330481 Истинное значение -116.8
МГУА -117.06264502762431 Греб.рег -154.93255237064943 Истинное значение -113.2
МГУА -106.87100828729282 Греб.рег -91.1079190313618 Истинное значение -113.0
МГУА -117.06264502762431 Греб.рег -89.5731902906906 Истинное значение -114.9
МГУА -86.48773480662983 Греб.рег -99.11025200193615 Истинное значение -66.54
МГУА -127.2542817679558 Греб.рег -73.68711191579712 Истинное значение -56.79
МГУА -106.87100828729282 Греб.рег -100.86484237246918 Истинное значение -137.5
МГУА -117.06264502762431 Греб.рег -110.66827240547786 Истинное значение -118.28
МГУА -106.87100828729282 Греб.рег -80.25815148676202 Истинное значение -119.24
МГУА -117.06264502762431 Греб.рег -110.89943110333274 Истинное значение -159.9
МГУА -127.2542817679558 Греб.рег -155.50076871738622 Истинное